In [26]:
import gc
import os

import numpy as np
import pandas as pd
import random
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [27]:
torch.__version__, torch.cuda.is_available()

('2.0.1+cu117', True)

In [28]:
pl.seed_everything(56, workers=True)

56

In [29]:
columns = ['variantid', 'main_pic_embeddings_resnet_v1', 'name_bert_64', 'color_parsed']
train_data = pd.read_parquet('./datasets/train_data.parquet', columns=columns).set_index('variantid')
test_data = pd.read_parquet('./datasets/test_data.parquet', columns=columns).set_index('variantid')
train_data

,main_pic_embeddings_resnet_v1,name_bert_64,color_parsed
variantid,,,
51195767,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485...",[оранжевый]
53565809,"[[1.1471839, -0.665361, 0.7745614, 0.26716197,...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897...",[красный]
56763357,"[[-0.90570974, 1.0296293, 1.0769907, 0.27746, ...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719...",None
56961772,"[[0.13133773, -0.5577079, 0.32498044, 0.191717...","[-0.44812852, 0.5283565, 0.28981736, -0.506841...",[черный]
61054740,"[[0.21696381, 0.10989461, -0.08012986, 0.69186...","[-0.72692573, 0.75206333, 0.37740713, -0.52502...",[черный]
...,...,...,...
820128810,"[[-1.4492652, -0.80129164, -0.12344764, 0.7194...","[-0.8253241, 0.6785133, 0.53978086, -0.4888316...",[пурпурный]
821135769,"[[0.012127608, -0.8534423, 0.5415518, -0.44912...","[-0.7413257, 0.46105132, 0.5639801, -0.5462132...",[черный]
822095690,"[[0.4248176, -0.15944786, -0.22844064, 0.42768...","[-0.49261805, 0.56726897, 0.7037877, -0.697246...",[черный]


In [30]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
train_pairs['target'] = train_pairs['target'].astype(int)
train_pairs

,target,variantid1,variantid2
0,0,51197862,51198054
1,1,53062686,536165289
2,1,53602615,587809782
3,1,53888651,89598677
4,0,56930698,551526166
...,...,...,...
306535,0,817327230,822083612
306536,0,817560551,818069912
306537,0,817854719,817857267
306538,0,820036017,820037019


In [31]:
name_labse_embs = pd.read_parquet('F:/name_labse_embs.parquet').set_index('variantid')
name_labse_embs

,name_labse_768
variantid,
51195767,"[-0.033874325, 0.03722446, 0.0029757991, 0.068..."
53565809,"[0.015568526, -0.03899538, 0.064447366, 0.0383..."
56763357,"[-0.033072222, -0.04237577, 0.020771954, 0.065..."
56961772,"[0.014727573, -0.025661988, 0.023943473, -0.00..."
61054740,"[0.043145332, -0.052424084, 0.017260496, 0.045..."
...,...
820128810,"[-0.003678058, -0.031628493, 0.0065589263, 0.0..."
821135769,"[-0.06858361, 0.027011767, -0.016400583, -0.02..."
822095690,"[-0.04474233, -0.034224413, 0.026076552, 0.026..."


In [32]:
color_vocab = {}
cur_id = 2
max_count = 0
for colors in np.concatenate([train_data['color_parsed'], test_data['color_parsed']]):
    if colors is None:
        continue
    max_count = max(max_count, len(colors))
    for value in colors:
        if value not in color_vocab:
            color_vocab[value] = cur_id
            cur_id += 1
len(color_vocab), max_count

(247, 20)

In [33]:
def color_to_idx(colors):
    if colors is None:
        return [1]
    return [color_vocab[color] for color in colors]

train_data['color_parsed'] = train_data['color_parsed'].apply(color_to_idx)
test_data['color_parsed'] = test_data['color_parsed'].apply(color_to_idx)
train_data['color_parsed']

variantid
51195767          [2]
53565809          [3]
56763357          [1]
56961772          [4]
61054740          [4]
               ...   
820128810        [17]
821135769         [4]
822095690         [4]
822101044         [4]
822394794    [28, 10]
Name: color_parsed, Length: 457063, dtype: object

In [34]:
from sklearn.model_selection import train_test_split

train_pairs, val_pairs = train_test_split(train_pairs, test_size=1/3, random_state=56)

In [35]:
class Args:
    batch_size = 128
    
args = Args()

In [36]:
def pad_colors(colors):
    max_len = 20
    if len(colors) > max_len:
        return colors[:max_len]
    return colors + [0] * (max_len - len(colors))

class ItemsDataset(Dataset):
    def __init__(self, pairs, data):
        super().__init__()
        self.pairs = pairs.values
        self.main_pic_embs = data['main_pic_embeddings_resnet_v1']
        self.name_embs = data['name_bert_64']
        self.name_labse_embs = name_labse_embs['name_labse_768']
        self.colors = data['color_parsed']
        self.pairs_len = len(self.pairs)
        
    def __len__(self):
        return self.pairs_len

    def __getitem__(self, idx):
        target, id1, id2 = self.pairs[idx, :]
        
        return (
            torch.tensor(np.concatenate([self.main_pic_embs[id1][0], self.name_embs[id1], self.name_labse_embs[id1]])),
            torch.tensor(pad_colors(self.colors[id1])),
            len(self.colors[id1]),
            torch.tensor(np.concatenate([self.main_pic_embs[id2][0], self.name_embs[id2], self.name_labse_embs[id2]])),
            torch.tensor(pad_colors(self.colors[id2])),
            len(self.colors[id2]),
            target
        )

In [37]:
def get_data_loader(pairs, data, batch_size, drop_last, shuffle):
    dataset = ItemsDataset(pairs, data)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=0,
        drop_last=drop_last,
        shuffle=shuffle,
        pin_memory=True
    )
    return data_loader

In [38]:
def get_loaders(args):
    train_loader = get_data_loader(
        pairs=train_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=True
    )
    
    val_loader = get_data_loader(
        pairs=val_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=False,
        shuffle=False
    )
    return train_loader, val_loader

In [39]:
train_loader, val_loader = get_loaders(args)
len(train_loader), len(val_loader)

(1596, 799)

In [40]:
for x1, colors1, colors_len1, x2, colors2, colors_len2, target in train_loader:
    print(x1, colors1, colors_len1, x2, colors2, colors_len2, target)
    break

tensor([[ 0.1161, -1.0675,  0.3716,  ..., -0.0537, -0.0297,  0.0599],
        [-0.0278,  0.0919,  0.4331,  ..., -0.0140, -0.0102, -0.0098],
        [ 0.2479, -0.3031,  0.0908,  ..., -0.0060, -0.0081,  0.0303],
        ...,
        [-0.2100, -0.5099,  0.3891,  ...,  0.0434, -0.0274, -0.0064],
        [ 0.6944,  0.5330, -1.5235,  ...,  0.0318, -0.0834,  0.0291],
        [ 0.1003, -0.4157, -0.1716,  ...,  0.0218,  0.0101,  0.0634]]) tensor([[ 5,  0,  0,  ...,  0,  0,  0],
        [20, 21,  0,  ...,  0,  0,  0],
        [ 3,  0,  0,  ...,  0,  0,  0],
        ...,
        [21,  0,  0,  ...,  0,  0,  0],
        [ 1,  0,  0,  ...,  0,  0,  0],
        [ 4,  0,  0,  ...,  0,  0,  0]]) tensor([1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1,
        1, 1, 1, 3, 1, 4, 3, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 1, 1, 9, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 4, 1, 2, 2, 2, 1, 1,
        1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1

In [90]:
class Net(pl.LightningModule):
    margin = 0.75
    
    def __init__(self):
        super().__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=len(color_vocab) + 2, 
            embedding_dim=(len(color_vocab) + 2) // 2, 
            padding_idx=0
        )
        self.lstm_hidden = 64
        self.lstm = nn.LSTM(
            input_size=(len(color_vocab) + 1) // 2, 
            hidden_size=self.lstm_hidden, 
            num_layers=1, 
            batch_first=True,
            bidirectional=True
        )
        
        # self.linear = nn.Linear(128 + 64, 256)
        
        features_num = 128 + 64 + 768 + self.lstm_hidden*2
        # output_size = 256
        # embedding_size = (features_num + output_size) // 2
        embedding_size = 512
        self.neck = nn.Sequential(
            nn.BatchNorm1d(features_num),
            nn.Linear(features_num, embedding_size, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(embedding_size),
            nn.Linear(embedding_size, embedding_size, bias=False),
            nn.BatchNorm1d(embedding_size),
        )
        # self.linear = nn.Linear(embedding_size, output_size)
        
    def forward(self, x, colors, colors_len):
        colors_emb = self.embedding(colors)
        colors_emb = pack_padded_sequence(colors_emb, colors_len.detach().cpu(), batch_first=True, enforce_sorted=False)
        output, (ht, ct) = self.lstm(colors_emb)
        output, _ = pad_packed_sequence(output, batch_first=True)
        # colors_output = output[:,-1,:]
        
        out_forward = output[range(len(output)), colors_len - 1, :self.lstm_hidden]
        out_reverse = output[:, 0, self.lstm_hidden:]
        colors_output = torch.cat([out_forward, out_reverse], 1)
        
        # return self.linear(x)
        # return self.linear(self.neck(x))
        x = torch.cat([x, colors_output], dim=1)
        return self.neck(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=1e-4, betas=(0.9, 0.999), weight_decay=0.05
        )
        # optimizer = torch.optim.SGD(self.parameters(), lr=1e-1, momentum=0.9, weight_decay=0.0001)
        return (
            {
                "optimizer": optimizer,
            },
        )
    
    def training_step(self, batch, batch_idx):
        # self.log('step', batch_idx, logger=True, on_epoch=True)
        x1, colors1, colors_len1, x2, colors2, colors_len2, labels = batch
        out1 = self.forward(x1, colors1, colors_len1)
        out2 = self.forward(x2, colors2, colors_len2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("train_loss", loss, on_step=False, logger=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):        
        x1, colors1, colors_len1, x2, colors2, colors_len2, labels = batch
        out1 = self.forward(x1, colors1, colors_len1)
        out2 = self.forward(x2, colors2, colors_len2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("val_loss", loss, logger=False, on_epoch=True, prog_bar=True)   
        
        try:
            auc = roc_auc_score(labels.detach().cpu(), 1 - dists.detach().cpu())
        except:
            auc = 0
            
        self.log("val_auc", auc, logger=False, on_epoch=True, prog_bar=True)
        
    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader
    
    def predict_step(self, batch, batch_idx):
        x1, colors1, colors_len1, x2, colors2, colors_len2, labels = batch
        out1 = self.forward(x1, colors1, colors_len1)
        out2 = self.forward(x2, colors2, colors_len2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        return torch.cat([out1, out2, (1 - dists).unsqueeze(-1)], dim=1).detach().cpu()

In [91]:
model = Net()

In [92]:
import torchinfo

torchinfo.summary(model)

Layer (type:depth-idx)                   Param #
Net                                      --
├─Embedding: 1-1                         30,876
├─LSTM: 1-2                              97,280
├─Sequential: 1-3                        --
│    └─BatchNorm1d: 2-1                  2,176
│    └─Linear: 2-2                       557,056
│    └─ReLU: 2-3                         --
│    └─BatchNorm1d: 2-4                  1,024
│    └─Linear: 2-5                       262,144
│    └─BatchNorm1d: 2-6                  1,024
Total params: 951,580
Trainable params: 951,580
Non-trainable params: 0

In [93]:
trainer = pl.Trainer(
    logger=False, # CSVLogger('./'),
    enable_checkpointing=False,
    
    accelerator='gpu', 
    devices=1,
    profiler='advanced',
    precision=32,
    check_val_every_n_epoch=1,
    max_epochs=10
)

trainer.fit(model)

Sanity Checking: 0it [00:00, ?it/s]

2023-05-23 03:56:40,848 [37012] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-23 03:56:40,965 [37012] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [94]:
test_preds = trainer.predict(model, val_loader)

2023-05-23 04:16:47,513 [37012] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Predicting: 0it [00:00, ?it/s]

In [95]:
roc_auc_score(val_pairs.target, np.concatenate([pred.numpy()[:, -1] for pred in test_preds]))

0.8756164858900239

In [96]:
train_dataset = ItemsDataset(train_pairs, train_data)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    num_workers=0,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [97]:
features = np.concatenate([pred.numpy() for pred in trainer.predict(model, train_loader)])

2023-05-23 04:17:15,061 [37012] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Predicting: 0it [00:00, ?it/s]

In [98]:
val_features = np.concatenate([pred.numpy() for pred in trainer.predict(model, val_loader)])

Predicting: 0it [00:00, ?it/s]

In [99]:
from catboost import CatBoostClassifier, Pool, cv

2023-05-23 04:18:40,531 [37012] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index



In [100]:
train_pool = Pool(
    data=features,
    label=train_pairs.target,
)

val_pool = Pool(
    data=val_features,
    label=val_pairs.target,
)

In [101]:
params = {
    'loss_function': 'Logloss',
    'custom_metric': ['AUC'],
    'task_type': 'CPU',
}

In [102]:
model_cb = CatBoostClassifier(**params, random_seed=56)
model_cb.fit(train_pool, eval_set=val_pool, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [103]:
np.max(model_cb.get_evals_result()['validation']['AUC'])

0.8797019040150235

In [104]:
from sklearn.metrics import precision_recall_curve, auc

precision, recall, thrs = precision_recall_curve(val_pairs.target, model_cb.predict_proba(val_pool)[:, 1])
gt_prec_level_idx = np.where(precision >= 0.75)[0]
auc(recall[gt_prec_level_idx], precision[gt_prec_level_idx])

0.7473427418378805